In [37]:
import glob # lista plików
import xml.etree.cElementTree as ET # przechodzenie po XMLu
import pandas as pd # data frame, zapisywanie danych do CSV
import numpy as np # histogram 2D
from tqdm import tqdm # pasek postępu - żeby było wiadomo że coś się dzieje ;)
 
import matplotlib.pyplot as plt # wykresy
import matplotlib.cm as cm # palety kolorów

In [52]:
def parse_gpx_file(filename: str) -> pd.DataFrame:
    """Funkcja wczytuje podany plik GPX i zwraca dataframe z jego zawartością"""
    
    def find_tag(xml_node: ET.Element, tag: str) -> ET.Element:
        """Funkcja szuka w XMLu node'a o określonym tagu. Zwraca znaleziony node lub None"""
        found = False
        for node in xml_node:
            if node.tag.endswith(tag):
                found  = True
                break
        
        if found:
            return node
 
        return None
 
    # wczytujemy plik XML
    tree = ET.parse(filename)
    root = tree.getroot()
 
    # począwszy od roota szukamy sekcji "trk"
    trk_node = find_tag(root, "trk")
    if not trk_node:
        return pd.DataFrame()
 
    # mając sekcję "trk" szukamy w niej "trkseg"
    trkseg_node = find_tag(trk_node, "trkseg")
    if not trkseg_node:
        return pd.DataFrame()
 
    # w sekcji "trkseg" mamy kolekcję punktów na trasie
    points = []
    for node in trkseg_node:
        # z każdego punktu wyciągamy interesujące nas informacje i dokładamy do listy
        elem = node.attrib
        for nn in node:
            elem[nn.tag.split("}")[-1]] = nn.text
        points.append(elem)
        
    # z listy robimy data frame
    points_df = pd.DataFrame(points)
 
    # korekcja typów zebranych danych
    points_df['lat'] = points_df['lat'].astype(float)
    points_df['lon'] = points_df['lon'].astype(float)
    points_df['ele'] = points_df['ele'].astype(float)
    points_df['time'] = points_df['time'].apply(lambda s: pd.to_datetime(s))
    points_df['filename'] = filename
 
    return points_df[['time', 'filename', 'lon', 'lat', 'ele']]

In [53]:
!gzip -d strava_archive/activities/*.gpx.gz

fish: No matches for wildcard “strava_archive/activities/*.gpx.gz”. See `help expand`.
gzip -d strava_archive/activities/*.gpx.gz
        ^


In [54]:
activities_data_gpx = "strava_archive/activities/*.gpx"
activities_data = "strava_archive/activities.csv"
activities_data_output = "strava_archive/activities_data.csv"

In [55]:
full_df = pd.DataFrame()
 
# listujemy pliki i każdy z nich parsujemy
for gpx_file in tqdm(glob.glob(activities_data_gpx)):
    tmp_df = parse_gpx_file(gpx_file)
 
    # wynik parsowania doklejamy do pełnego data frame
    full_df = pd.concat([full_df, tmp_df], axis=0)

100%|█████████████████████████████████████████| 161/161 [00:13<00:00, 11.85it/s]


In [ ]:
gpx_df = full_df

In [78]:
import pickle 
with open('fit2gpx_df.pkl','rb') as f:
    fit_df = pickle.load(f)

In [80]:
full_df = pd.concat([full_df, fit_df], axis = 0)

In [81]:
import folium
from folium.plugins import HeatMap


In [82]:
agg_points = full_df[['lon', 'lat', 'filename']].copy()
agg_points['lon'] = agg_points['lon'].apply(lambda x: round(x, 4))
agg_points['lat'] = agg_points['lat'].apply(lambda x: round(x, 4))
 
agg_points_plot = agg_points.groupby(['lat', 'lon']).nunique().reset_index()


In [83]:
centrum = (np.mean(agg_points_plot['lat']), np.mean(agg_points_plot['lon']))
 
# budujemy obiekt Map będący początkowo tylko podkładem z mapą
m = folium.Map(
    location =centrum,
    tiles="OpenStreetMap",
    zoom_start=10,
    zoom_control=True
)
 
# wyliczamy heatmapę i dodajemy ją do obiektu typu Map
HeatMap(agg_points_plot.values.tolist(), radius=10).add_to(m)

In [84]:
m.save("heatmap.html")